# Transcribe a YouTube Video in English with Assembly AI and yt-dlp

### References:

- [Assembly AI documentation](https://www.assemblyai.com/docs)
- [yt-dlp on GitHub](https://github.com/yt-dlp/yt-dlp)

## Preparation

#### Imports and Globals

In [1]:
import assemblyai as aai
import yt_dlp
import json

from config import *

aai.settings.api_key = aai_key
YT_BASE_URL = 'https://www.youtube.com/watch?v='
DST_FOLDER = 'files'

#### Task-specific Variables

In [2]:
# title = 'The Power To OverRule G-d'  # actually not needed as yt-dlp can obtain it from the video
v_id = 'wKS2sFRI87w'  # the main identifier of the video, absolutely needeed

#### Pull and save the soundtrack with yt-dlp

In [24]:
url = f'{YT_BASE_URL}{v_id}'

ydl_opts = {
    'format': 'm4a/bestaudio/best',  # The best audio version in m4a format
    'outtmpl': f'{DST_FOLDER}/%(title)s_%(id)s.%(ext)s',  
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    info = ydl.extract_info(url)
    audio_file = ydl.prepare_filename(info)

print(f'\n>>> Downloaded to: {audio_file}')

[youtube] Extracting URL: https://www.youtube.com/watch?v=wKS2sFRI87w
[youtube] wKS2sFRI87w: Downloading webpage
[youtube] wKS2sFRI87w: Downloading tv client config
[youtube] wKS2sFRI87w: Downloading player c548b3da
[youtube] wKS2sFRI87w: Downloading tv player API JSON
[youtube] wKS2sFRI87w: Downloading ios player API JSON
[youtube] wKS2sFRI87w: Downloading m3u8 information
[info] wKS2sFRI87w: Downloading 1 format(s): 140
[download] files\The Power To OverRule G-d：_wKS2sFRI87w.m4a has already been downloaded
[download] 100% of   23.92MiB

>>> Downloaded to: files\The Power To OverRule G-d：_wKS2sFRI87w.m4a


## Building the transcript via AssemblyAI API

In [ ]:
config = aai.TranscriptionConfig(language_detection=True)

In [27]:
transcriber = aai.Transcriber()
transcript = transcriber.transcribe(audio_file, config)
print(transcript.status)

TranscriptStatus.completed


In [20]:
file_name = audio_file.split("\\")[-1].split(".")[0]
file_name

'The Power To OverRule G-d：_wKS2sFRI87w'

In [21]:
json.dump(transcript.json_response, 
          open(f'files/transcript_{file_name}_{transcript.id}.json', 'w', encoding='utf-8'), 
          indent=4, 
          ensure_ascii=False
)

with open(f'files/transcript_{file_name}_{transcript.id}.txt', 'w', encoding='utf-8') as f:
    f.write(transcript.text)

## Working with the transcript

In [ ]:
# load the transcript by id if necessary
job_id = '1dc162c7-0114-4698-be0a-8259ad1e7edf'

transcript = aai.Transcript.get_by_id(job_id)
transcript.status

<TranscriptStatus.completed: 'completed'>

### Search for word

In [23]:
def convert_millis(millis):
    seconds = millis // 1000
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{hours:02}:{minutes:02}:{seconds:02}"

def find_words(query):
    matches = transcript.word_search(query.split())
    for match in matches:
        print(f'{match.text}: ', end='')
        print('; '.join([convert_millis(start) for start, end in match.timestamps]))

In [24]:
query = 'Mode Cotton'
find_words(query)

mode: 00:14:45
cotton: 00:14:46


In [12]:
matches = transcript.word_search(['God'])
matches

[WordSearchMatch(text='god', count=62, timestamps=[(20664, 20984), (156172, 156800), (159174, 159366), (161486, 161686), (170430, 170646), (190434, 190666), (234720, 234984), (324018, 324274), (329794, 329986), (394814, 395046), (401282, 401466), (405602, 405834), (414562, 414746), (415762, 415946), (416946, 417194), (431460, 431932), (437780, 438044), (492546, 492746), (494514, 494890), (527540, 527844), (538984, 539360), (552664, 552944), (574656, 574856), (600348, 600644), (694018, 694178), (695754, 695986), (696666, 696818), (701882, 702370), (724746, 725106), (730666, 730914), (773528, 773728), (861360, 861664), (864048, 864256), (870208, 870416), (988096, 988296), (991316, 991468), (1200312, 1200448), (1236932, 1237560), (1242332, 1242516), (1252796, 1253092), (1269166, 1269542), (1275086, 1275334), (1279470, 1279686), (1283412, 1283596), (1294276, 1294476), (1295300, 1295468), (1310908, 1311116), (1321956, 1322156), (1359460, 1359724), (1361540, 1361756), (1366496, 1366648), (13